# Read Delta files 


**Autoloader set up**  
This Notebook sets up an Autoloader job that runs on a manual trigger to collect ack messages from the ack eventhubs


## Option 01: Using Oauth Approach

In [0]:
from pyspark.sql.functions import *

In [0]:
config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env_name = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()

print(f"env_code: {lz_key}")  # This won't be redacted
print(f"env_name: {env_name}")  # This won't be redacted

In [0]:

KeyVault_name = f"ingest{lz_key}-meta002-{env_name}"

# Service principal credentials
client_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-ID")
client_secret = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-SECRET")
tenant_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-TENANT-ID")

# Storage account names
curated_storage = f"ingest{lz_key}curated{env_name}"
checkpoint_storage = f"ingest{lz_key}xcutting{env_name}"

# Spark config for curated storage (Delta table)
spark.conf.set(f"fs.azure.account.auth.type.{curated_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{curated_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{curated_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{curated_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{curated_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{checkpoint_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{checkpoint_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{checkpoint_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{checkpoint_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{checkpoint_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


In [0]:
dbutils.fs.ls(f"abfss://bronze@ingest00curatedsbox.dfs.core.windows.net")

In [0]:
df_audit = spark.read.format("delta").load(f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ARM/AUDIT/TD/td_cr_audit_table")
display(df_audit)

## Option 02: Using Mounts Approach

In [0]:
dbutils.fs.ls(f"/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/TD/td_cr_audit_table")
# dbutils.fs.ls(f"/mnt/")

In [0]:
df_audit = spark.read.format("delta").load("/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/TD/td_cr_audit_table")
display(df_audit)

In [0]:
dbutils.notebook.exit("Notebook execution completed successfully.")

## Appendix

In [0]:
dbutils.fs.ls("/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/TD/td_cr_audit_table")


In [0]:
# dbutils.fs.ls("/mnt/ingest00curatedsboxsilver/ARIADM/")

In [0]:
# dbutils.fs.ls("/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT")

In [0]:
# dbutils.fs.rm("/mnt/autoLoaderSchema/ARMTD/ACK/ack", recurse=True)

In [0]:
# display(spark.read.format("delta").load("/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/ARIATD/td_ack_audit"))

In [0]:
# dbutils.fs.ls("/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/ARIATD/td_ack_audit/")

In [0]:
# spark.read.format("delta").load("/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/ARIATD/td_ack_audit").display()

In [0]:
# dbutils.fs.ls("/mnt/autoLoaderSchema/ARMTD/ACK/ack")

In [0]:
# dbutils.fs.rm("/mnt/autoLoaderSchema/ARMTD/ACK/ack", True)
# dbutils.fs.rm("/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/ARIATD/", True)

In [0]:
# Define the paths to the tables
audit_delta_path = "/mnt/ingest00curatedsboxsilver//ARIADM/ARM/AUDIT/TD/td_cr_audit_table"
ack_path = "/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/TD/td_ack_audit_table"
output_subdir_input_upload = "/mnt/ingest00curatedsboxsilver/ARIADM/ARM/response/TD/input_upload"
output_subdir_create_record_upload = "/mnt/ingest00curatedsboxsilver/ARIADM/ARM/response/TD/create_record"
output_subdir_upload_file_upload = "/mnt/ingest00curatedsboxsilver/ARIADM/ARM/response/TD/upload_file"
output_subdir_amalgamated_responses = "/mnt/ingest00curatedsboxsilver/ARIADM/ARM/response/TD/amalgamated_responses"

# Read the Delta table for joh_cr_audit_table
df_audit = spark.read.format("delta").load(audit_delta_path)

# Read the Delta table for joh_ack_audit
# df_ack = spark.read.format("delta").load(ack_path)

# Read the response data for input_upload
# df_input_upload = spark.read.format("parquet").load(output_subdir_input_upload)

# # Read the response data for create_record_upload
# df_create_record_upload = spark.read.format("parquet").load(output_subdir_create_record_upload)

# # Read the response data for upload_file
# df_upload_file_upload = spark.read.format("parquet").load(output_subdir_upload_file_upload)


# Read the response data for df_amalgamated_responses
# df_amalgamated_responses = spark.read.format("delta").load(output_subdir_amalgamated_responses)

# Display the DataFrames using Databricks display
display(df_audit)
display(df_ack)
# display(df_input_upload)
# display(df_create_record_upload)
# display(df_upload_file_upload)
# display(df_amalgamated_responses)